<a href="https://colab.research.google.com/github/yektaKamane/GPU_Programming_Course/blob/main/HW4/HW_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW4 - class
The implementation of min and max using reduce algorithm


## Original version
The code below I found in the internet and I'm using it as a starting pint to write my own code.
[src link](https://github.com/MaxKotlan/Cuda-Find-Max-Using-Parallel-Reduction)

In [ ]:
%%cu
#include <stdio.h>
#include <cuda.h>
#include <time.h>

#define MAX_CUDA_THREADS_PER_BLOCK 1024

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

struct Startup{
    int random_range = INT_MAX;
    int threads_per_block = MAX_CUDA_THREADS_PER_BLOCK;
} startup;

struct DataSet{
    float* values;
    int  size;
};

struct Result{
    float MaxValue;
    float KernelExecutionTime;
};

DataSet generateRandomDataSet(int size){
    DataSet data;
    data.size = size;
    data.values = (float*)malloc(sizeof(float)*data.size);

    for (int i = 0; i < data.size; i++)
        data.values[i] = (float)(rand()%startup.random_range);

    return data;
}

__global__ void Max_Interleaved_Addressing_Global(float* data, int data_size){
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx < data_size){
        for(int stride=1; stride < data_size; stride *= 2) {
            if (idx % (2*stride) == 0) {
                float lhs = data[idx];
                float rhs = data[idx + stride];
                data[idx] = lhs < rhs ? rhs : lhs;
            }
            __syncthreads();
        }
    }
}

__global__ void Max_Interleaved_Addressing_Shared(float* data, int data_size){
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    __shared__ float sdata[MAX_CUDA_THREADS_PER_BLOCK];
    if (idx < data_size){

        /*copy to shared memory*/
        sdata[threadIdx.x] = data[idx];
        __syncthreads();

        for(int stride=1; stride < blockDim.x; stride *= 2) {
            if (threadIdx.x % (2*stride) == 0) {
                float lhs = sdata[threadIdx.x];
                float rhs = sdata[threadIdx.x + stride];
                sdata[threadIdx.x] = lhs < rhs ? rhs : lhs;
            }
            __syncthreads();
        }
    }
    if (idx == 0) data[0] = sdata[0];
}


__global__ void Max_Sequential_Addressing_Shared(float* data, int data_size){
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    __shared__ float sdata[MAX_CUDA_THREADS_PER_BLOCK];
    if (idx < data_size){

        /*copy to shared memory*/
        sdata[threadIdx.x] = data[idx];
        __syncthreads();

        for(int stride=blockDim.x/2; stride > 0; stride /= 2) {
            if (threadIdx.x < stride) {
                float lhs = sdata[threadIdx.x];
                float rhs = sdata[threadIdx.x + stride];
                sdata[threadIdx.x] = lhs < rhs ? rhs : lhs;
            }
            __syncthreads();
        }
    }
    if (idx == 0) data[0] = sdata[0];
}

/*Algorithm Information. Includes pointers to different kernels, so they can be executed dynamically*/
const int Algorithm_Count = 3;
typedef void (*Kernel)(float *, int);
const char* Algorithm_Name[Algorithm_Count]= {"Max_Interleaved_Addressing_Global", "Max_Interleaved_Addressing_Shared", "Max_Sequential_Addressing_Shared"};
const Kernel Algorithm[Algorithm_Count]    = { Max_Interleaved_Addressing_Global,   Max_Interleaved_Addressing_Shared,   Max_Sequential_Addressing_Shared};

Result calculateMaxValue(DataSet data, Kernel algorithm){
    float* device_data;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);    

    gpuErrchk(cudaMalloc((void **)&device_data,  sizeof(float)*data.size));
    gpuErrchk(cudaMemcpy(device_data, data.values, sizeof(float)*data.size, cudaMemcpyHostToDevice));


    int threads_needed = data.size;
    cudaEventRecord(start);
    algorithm<<< threads_needed/ startup.threads_per_block + 1, startup.threads_per_block>>>(device_data, data.size);
    cudaEventRecord(stop);
    gpuErrchk(cudaEventSynchronize(stop));

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    float max_value;
    gpuErrchk(cudaMemcpy(&max_value, device_data, sizeof(float), cudaMemcpyDeviceToHost));
    gpuErrchk(cudaFree(device_data));

    Result r = {max_value, milliseconds};
    return r;
}

Result calculateMaxValue(DataSet data){
    return calculateMaxValue(data, Algorithm[Algorithm_Count - 1]);
}

void printDataSet(DataSet data){
    for (int i = 0; i < data.size; i++)
        printf("%.6g, ", data.values[i]);
    printf("\n");
}

void benchmarkCSV(){
    /*Print Headers*/
    printf("Elements, ");
    for (int algoID = 0; algoID < Algorithm_Count; algoID++)
        printf("%s, ", Algorithm_Name[algoID]);
    printf("\n");
    /*Benchmark*/
    for (int dataSize = 2; dataSize < INT_MAX; dataSize*=2){
        DataSet random = generateRandomDataSet(dataSize);
        printf("%d, ", dataSize);
        for (int algoID = 0; algoID < Algorithm_Count; algoID++) {
            Result r = calculateMaxValue(random, Algorithm[algoID]);
            printf("%g, ", r.KernelExecutionTime);
        }
        printf("\n");
        free(random.values);
    }
}

int main(int argc, char** argv){
    srand(time(nullptr));
    benchmarkCSV();
}

tcmalloc: large alloc 1073741824 bytes == 0x55e576c82000 @  0x7f824edfb1e7 0x55e574bbb167 0x55e574bbb53f 0x55e574bbb618 0x7f824de2cbf7 0x55e574bbb02a
tcmalloc: large alloc 2147483648 bytes == 0x55e576c82000 @  0x7f824edfb1e7 0x55e574bbb167 0x55e574bbb53f 0x55e574bbb618 0x7f824de2cbf7 0x55e574bbb02a
tcmalloc: large alloc 4294967296 bytes == 0x55e576c82000 @  0x7f824edfb1e7 0x55e574bbb167 0x55e574bbb53f 0x55e574bbb618 0x7f824de2cbf7 0x55e574bbb02a
tcmalloc: large alloc 18446744065119617024 bytes == (nil) @  0x7f824edfb1e7 0x55e574bbb167 0x55e574bbb53f 0x55e574bbb618 0x7f824de2cbf7 0x55e574bbb02a
GPUassert: out of memory /tmp/tmpr3mxsagt/43c50341-76f8-41e9-ae40-08b081c94125.cu 112
Elements, Max_Interleaved_Addressing_Global, Max_Interleaved_Addressing_Shared, Max_Sequential_Addressing_Shared, 
2, 0.12048, 0.021696, 0.021312, 
4, 0.016416, 0.02032, 0.01776, 
8, 0.01552, 0.020896, 0.017536, 
16, 0.015168, 0.020864, 0.017664, 
32, 0.01664, 0.02064, 0.017504, 
64, 0.01872, 0.033184, 0.017952,

## My version

**Output file #1:**
* The number of tests(5 for: 1M, 5M, 10M, 15M, 20M)
* Each test's size and runtime

**Output file #2:**
* The number of tests
* Each test's generated numbers, result of max/min function, runtime and size






In [ ]:
%%cu

#include <stdio.h>
#include <cuda.h>
#include <time.h>


#define MAX_CUDA_THREADS_PER_BLOCK 1024

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true){
   if (code != cudaSuccess) {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

struct Startup{
    int random_range = INT_MAX;
    int threads_per_block = MAX_CUDA_THREADS_PER_BLOCK;
} startup;

struct DataSet{
    int* values;
    int  size;
};

struct Result{
    int MaxValue;
    float KernelExecutionTime;
};

DataSet generateRandomDataSet(int size){
    DataSet data;
    data.size = size;
    data.values = (int*)malloc(sizeof(int)*data.size);

    for (int i = 0; i < data.size; i++)
        data.values[i] = (int)(rand()%startup.random_range);

    return data;
}

__global__ void Max_Sequential_Addressing_Shared(int* data, int data_size){
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    __shared__ int sdata[MAX_CUDA_THREADS_PER_BLOCK];
    if (idx < data_size){

        /*copy to shared memory*/
        sdata[threadIdx.x] = data[idx];
        __syncthreads();

        for(int stride=blockDim.x/2; stride > 0; stride /= 2) {
            if (threadIdx.x < stride) {
                int lhs = sdata[threadIdx.x];
                int rhs = sdata[threadIdx.x + stride];
                sdata[threadIdx.x] = lhs < rhs ? rhs : lhs;
            }
            __syncthreads();
        }
    }
    if (idx == 0) data[0] = sdata[0];
}


Result calculateMaxValue(DataSet data){
    int* device_data;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);    

    gpuErrchk(cudaMalloc((void **)&device_data,  sizeof(int)*data.size));
    gpuErrchk(cudaMemcpy(device_data, data.values, sizeof(int)*data.size, cudaMemcpyHostToDevice));


    int threads_needed = data.size;
    cudaEventRecord(start);
    Max_Sequential_Addressing_Shared<<< threads_needed/ startup.threads_per_block + 1, startup.threads_per_block>>>(device_data, data.size);
    cudaEventRecord(stop);
    gpuErrchk(cudaEventSynchronize(stop));

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    int max_value;
    gpuErrchk(cudaMemcpy(&max_value, device_data, sizeof(int), cudaMemcpyDeviceToHost));
    gpuErrchk(cudaFree(device_data));

    Result r = {max_value, milliseconds}; // this might cause error
    return r;
}


void printDataSet(DataSet data){
    for (int i = 0; i < data.size; i++)
        printf("%d, ", data.values[i]);
    printf("\n");
}


void benchmarkCSV(){
    /*Benchmark*/
    FILE *out_file = fopen("Results", "w");

    int size[] = {1, 5, 10, 15, 20};
    for (int i = 0; i<5; i++){
        int dataSize = size[i]*1000000;
        DataSet random = generateRandomDataSet(dataSize);
        Result r = calculateMaxValue(random);

        fprintf(out_file, "Data size: %d\n", dataSize);
        //fprintf(out_file, "Maximum value: %d\n", r.MaxValue);
        fprintf(out_file, "Execution time: %f\n----\n", r.KernelExecutionTime);

        printf("%d, ", dataSize);
        printf("%g, ", r.KernelExecutionTime);
        printf("\n");
        free(random.values);
    }
}


int main(int argc, char** argv){
    srand(time(nullptr));
    benchmarkCSV();
}



1000000, 0.486816, 
5000000, 1.78016, 
10000000, 3.53635, 
15000000, 5.29286, 
20000000, 7.06013, 



In [ ]:
! ls
! nvcc --version
! nvcc -o max-reduce max-reduce.cu

cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb  Results	src
max-reduce.cu					   sample_data
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
! ./max-reduce

1000000, 0.488224, 
5000000, 1.7848, 
10000000, 3.53549, 
15000000, 5.30483, 
20000000, 7.05405, 


In [ ]:
! nvprof ./max-reduce

==18669== NVPROF is profiling process 18669, command: ./max-reduce
==18669== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the recommended nvcc version 
1000000, 0.56912, 
5000000, 1.84294, 
10000000, 3.67098, 
15000000, 5.49165, 
20000000, 7.31232, 
==18669== Profiling application: ./max-reduce
==18669== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   60.23%  28.219ms         5  5.6439ms  554.87us  10.934ms  [CUDA memcpy HtoD]
                   39.74%  18.619ms         5  3.7239ms  370.40us  7.2974ms  Max_Sequential_Addressing_Shared(int*, int)
                    0.03%  16.288us         5  3.2570us  2.8800us  3.5520us  [CUDA memcpy DtoH]
      API calls:   83.22%  257.23ms        10  25.723ms     764ns  257.16ms  cuda